In [9]:
#read how many imagaes are there in a folder called raw/images/all_objects

#create pandas dataframe with below columns
#image_name, image_path, col_number, row_number, label
#Take col_number and row_number starting from 1,1 at top left corner and 
# increasing to right and down respectively till 8,8
import os
import pandas as pd
from PIL import Image, ImageDraw

def draw_grid_on_image(image_path, tile_width, tile_height, output_path):
    # Open the image
    img = Image.open(image_path)
    draw = ImageDraw.Draw(img)
    img_width, img_height = img.size

    # Draw vertical lines
    for x in range(0, img_width, tile_width):
        draw.line([(x, 0), (x, img_height)], fill='red', width=2)

    # Draw horizontal lines
    for y in range(0, img_height, tile_height):
        draw.line([(0, y), (img_width, y)], fill='red', width=2)
    
    # Add numbers to each tile starting from (1,1) at top-left
    num_tiles_x = img_width // tile_width
    num_tiles_y = img_height // tile_height
    for i in range(num_tiles_x):
        for j in range(num_tiles_y):
            x = i * tile_width + tile_width // 2
            y = j * tile_height + tile_height // 2            
            draw.text((x+1, y+1), f'{j+1},{i+1}', fill='blue')    

    # Save the image with grid
    img.save(output_path)

image_folder = '../data/raw/images/all_objects'
output_grid_image_path = '../data/processed/tiled_images'
data = []
for image_name in os.listdir(image_folder):
    if image_name.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_folder, image_name)
        output_path = os.path.join(output_grid_image_path, image_name)        
        draw_grid_on_image(image_path, 100, 75, output_path)
        for col in range(8):
            for row in range(8):                
                data.append({
                    'image_name': image_name,                    
                    'col_number': col + 1,
                    'row_number': row + 1,
                    'label': 1
                })
df = pd.DataFrame(data)
print(df.head())
print(f'Total records: {len(df)}')
#save dataframe to csv
df.to_csv('../data/processed/image_tiles_labels.csv', index=False)

      image_name  col_number  row_number  label
0  cricket_1.jpg           1           1      1
1  cricket_1.jpg           1           2      1
2  cricket_1.jpg           1           3      1
3  cricket_1.jpg           1           4      1
4  cricket_1.jpg           1           5      1
Total records: 768


PermissionError: [Errno 13] Permission denied: '../data/processed/image_tiles_labels.csv'